In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
import sqlite3

In [53]:
conn = sqlite3.connect('Performance.db')
c = conn.cursor()
slippage = 0.002

In [74]:
df = pd.read_sql_query("SELECT * FROM SVM30", conn, parse_dates = 'Starttime' )
df = df.drop('ID', 1)
#df['Starttime'] = df['Starttime'].str.split('+').str[0]
#df['Starttime'] = pd.to_datetime(df['Starttime'], format="%Y-%m-%d %H:%M:%Sz")
df.set_index('Starttime', inplace = True)
t0 = df.index[0]
t1 = df.index[-1]
idx = pd.date_range(start = t0 , end = t1, freq = '30T')
#print(idx)
df = df.reindex(idx)
df['diff'] = df['Price'].shift(-1) - df['Price']
df['actual'] = df['diff'] >0
df['actual'] = df['actual'].astype(int)
df.dropna(axis = 0, how = 'any', inplace = True)
df['buysell'] = df['Prediction']*2 -1
df['pnl'] = df['buysell']*df['Prob']*df['diff']
df['fees'] = (df['Prob'])*df['Price']*slippage
df['netpnl'] = df['pnl'] - df['fees']
numright = df[df['Prediction'] == df['actual']].shape[0]
numtotal = df.shape[0]

df.head(30)

,Price,Prediction,Prob,diff,actual,buysell,pnl,fees,netpnl
2017-10-15 23:00:00,280.93,0.0,0.568066,0.06,1,-1.0,-0.034084,0.319173,-0.353257
2017-10-15 23:30:00,280.99,1.0,0.666199,0.83,1,1.0,0.552945,0.374390,0.178555
2017-10-16 00:00:00,281.82,0.0,0.665359,1.39,1,-1.0,-0.924848,0.375023,-1.299871
2017-10-16 00:30:00,283.21,0.0,0.912387,1.55,1,-1.0,-1.414199,0.516794,-1.930994
2017-10-16 02:30:00,289.84,0.0,0.888841,0.16,1,-1.0,-0.142215,0.515243,-0.657458
2017-10-16 03:00:00,290.00,0.0,0.883029,-2.71,0,-1.0,2.393008,0.512157,1.880852
2017-10-16 03:30:00,287.29,1.0,0.966226,-0.05,0,1.0,-0.048311,0.555174,-0.603486
2017-10-16 04:00:00,287.24,1.0,0.762256,0.02,1,1.0,0.015245,0.437901,-0.422656
2017-10-16 04:30:00,287.26,1.0,0.876211,-3.41,0,1.0,-2.987881,0.503401,-3.491282
2017-10-16 05:00:00,283.85,1.0,0.907041,7.15,1,1.0,6.485342,0.514927,5.970415


In [72]:
print ("Total net pnl:" , df['netpnl'].sum())
print ("Numright" , numright , "numtotal" , numtotal)


('Total net pnl:', -18.702414583681346)
('Numright', 27, 'numtotal', 55)
